In [ ]:
set.seed(624) #ONLY WRITE THIS ONCE

library(tidyverse)
library(dplyr)
library(repr)
library(tidymodels)
library(repr)
library(tidymodels)

# Libraries with functions needed are loaded and the seed is set to allow for the analysis to be reproducible

In [ ]:
options(repr.matrix.max.rows= 8)

Expected_Years_of_Schooling_vs_HDI<- read_tsv("https://raw.githubusercontent.com/MarranaTT/Group_46_Final_Project/main/Expected_years%20of%20schooling_(years).tsv",
    col_names = FALSE, skip =6)%>%
    slice(1:(n()-9))%>%
        select(X1:X62)%>%
        filter(X1 != "NA")
Expected_Years_of_Schooling_vs_HDI


#The raw data was read in, and edited so that it had a structure from which we could start to make it tidy. 
# The data alterations included skipping/getting rid of rows, choosing only certain columns, 
# an excluding any values that were not pertinent to our analysis.

The data table presented above is the result of reading in our HDI and Years of Schooling Data from the respective data source. The data is not tidy because although each value is a single cell, each row is not a single observation and each column is not a single variable. 

The col_names argument was given the logical statement FALSE as there are no column names in the original dataframe. The last 9 rows were then omitted using the slice function, which was followed by selecting the first 62 columns of the dataframe, and then deleting any NA values found within the first row since this will ensure that only country names are found in the first row (no invalid "values"). 

In [ ]:
EYS_vs_HDI <- Expected_Years_of_Schooling_vs_HDI[, colSums(is.na(Expected_Years_of_Schooling_vs_HDI)) < nrow(Expected_Years_of_Schooling_vs_HDI)]

#This was done to get rid of all the columns that had NA as values for all the rows.

Invalid_Col <- c("X62")

#This was done to give the last column an object name so that it can be used in the following code:

EYS_vs_HDI_NA_Drop = EYS_vs_HDI[,!(names(EYS_vs_HDI) %in% Invalid_Col)]

EYS_vs_HDI_NA_Drop

#The result of this code returns a data frame with only 61 columns and no NA values.

Using the original, slightly altered data, we used a variation of functions which allowed us to get rid of any columns with NA values and also get rid of a column that had no relevance to our study. This column was omitted because it's values (letters) were used as a legend in the original data frame; we will not be using the legend in the analysis. 

In [ ]:
EYS_vs_HDI_Col_ShiftUp<- EYS_vs_HDI_NA_Drop[-c(1),]
#This was done to make the first row the column names

EYS_vs_HDI_Colnames_Created <- c("HDI_Rank","Name_of_Country","1990","1991", "1992", "1993", "1994","1995", "1996", "1997", "1998","1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007",  "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015","2016", "2017", "2018", "2019")
colnames(EYS_vs_HDI_Col_ShiftUp) <- EYS_vs_HDI_Colnames_Created
#This was done to give the columns column names

EYS_vs_HDI_Tidy <- pivot_longer(EYS_vs_HDI_Col_ShiftUp, cols = "1990":"2019",
                 names_to = "Years",
                values_to = "Average_Years_of_Schooling")%>%
        select("HDI_Rank", "Name_of_Country", "Years", "Average_Years_of_Schooling")%>%
        filter(HDI_Rank != "..", Average_Years_of_Schooling != ".." )%>%
        mutate(HDI_Rank = as.numeric(HDI_Rank))%>%
        mutate(Average_Years_of_Schooling = as.numeric(Average_Years_of_Schooling))%>%
                select(-Name_of_Country,-Years)
EYS_vs_HDI_Tidy

#This step tidies the data into 2 columns (each one represents a variable) by making the data frame longer 
#(from its wide original shape).
#It allows for the exclusion of rows with ".." values
#The HDI_Rank is changed into a numerical variable which allows for further analysis.
#The columns that do not contribute to the analysis are also omitted.

The data is now in a tidy format from which further analysis can be conducted. The resulting table has two columns: the first one being the dependent variable HDI Rank of a country as per UNDP (United Nations Development Programme) records, and the corresponding Average Years of schooling which is the independent variable of this analysis. Both of the columns are numerical values which allows for them to be visualized using a scatter plot. Additionally the three conditions for tidy data are met: each row is a single observation, each column is a single variable, and each value is a single cell.

In [ ]:
EYS_vs_HDI_Tidy_Range_Classification <- EYS_vs_HDI_Tidy %>%
                    mutate(ranges = cut(HDI_Rank,breaks = c(1, 66, 119, 156, 189),labels = c("Very_High","High","Medium", "Low"), include.lowest = TRUE)) %>%
                    group_by(ranges) %>%
                    summarise(Number_of_rows= n())%>%
                    rename(HDI_ranges = ranges)
EYS_vs_HDI_Tidy_Range_Classification

#Counts the number of rows (count of HDI ranks) that have a specific value within a given range. 

The resulting table groups the original data of HDI ranks into four ranges: very high, high, medium and low. It counts the number of rows with a specific HDI value that falls into the ranges which we created based on research. These ranges contribute to our understanding of the distribution of the data.

In [ ]:
EYS_vs_HDI_Split <- initial_split(EYS_vs_HDI_Tidy, prop = 0.75, strata = HDI_Rank)
EYS_vs_HDI_Split

EYS_vs_HDI_Train <- training(EYS_vs_HDI_Split)
EYS_vs_HDI_Train

EYS_vs_HDI_Test <- testing(EYS_vs_HDI_Split)
# The tidy data is split into training and testing sets so that regression models may be applied.

EYS_vs_HDI_Tidy_Summary <- summary(EYS_vs_HDI_Train)
EYS_vs_HDI_Tidy_Summary

# The summary results in a table that includes important information about the variables including the median and mean.

EYS_vs_HDI_Train_Count <- count(EYS_vs_HDI_Train)%>%
                        rename(Number_of_Observations = n)
EYS_vs_HDI_Train_Count

# This counts the rows in the training data so that we know how many observations we have for the analysis.

The information presented by the tables above furthers the statistical meaning of the data that was initially shown by grouping it into the four ranges. Using the mean and the median, a histogram could be created to explore in more detail the distribution, however this is not beneficial to our analysis. Regardless, it is important to acknowledge it. 

In [ ]:
options(repr.plot.width=24, repr.plot.height=24)

EYS_vs_HDI_Train_Plot <- EYS_vs_HDI_Train %>%
                    ggplot(aes(x = Average_Years_of_Schooling, y = HDI_Rank))+
                    geom_point()+
                    labs(x= "Level of Education of an Individual (Average Years of Schooling)" , y = "HDI Rank of a Country (1 to 189)")+
                    ggtitle("The Correlation between the Level of Education (Average Years of Schooling) \n and the HDI Rank of a Country (1 to 189)")+
                    theme(axis.text.x = element_text(angle = 90, , hjust = 1),
                        text = element_text(size= 20))+
                    theme(plot.title = element_text(hjust = 0.5)) +
                    scale_x_continuous(expand = c(0,0), limits = c(0, NA))+
                    scale_y_continuous(expand = c(0,0), limits = c(0, NA))
EYS_vs_HDI_Train_Plot

# This plots HDI ranks (y-axis) and Average Years of Schooling (x-axis) for exploratory analysis. 

In [ ]:
lm_spec <- linear_reg() %>%
           set_engine("lm") %>%
           set_mode("regression")
lm_spec

# Sets the linear regression model specification.

lm_recipe <- recipe(HDI_Rank ~ Average_Years_of_Schooling, data = EYS_vs_HDI_Train)

lm_fit <- workflow() %>%
          add_recipe(lm_recipe) %>%
          add_model(lm_spec) %>%
          fit(data = EYS_vs_HDI_Train)
lm_fit

# Creates recipe, establishes a workflow analysis and fits the linear regression model for our training dataset. 
# The equation of the line of best fit can be found based on this step of the data analysis (using Coefficients).

In [ ]:
lm_predictions <- EYS_vs_HDI_Train %>%
    ggplot(aes(x = Average_Years_of_Schooling, y = HDI_Rank)) +
        geom_point() +          
        geom_smooth(method = "lm", se = FALSE) +
        labs(x= "Level of Education of an Individual (Average Years of Schooling)" , y = "HDI Rank of a Country (1 to 189)")+
                    ggtitle("The Correlation between the Level of Education (Average Years of Schooling) \n and the HDI Rank of a Country (1 to 189)")+
                    theme(axis.text.x = element_text(angle = 90, , hjust = 1),
                        text = element_text(size= 20))+
                    theme(plot.title = element_text(hjust = 0.5)) +
                    scale_x_continuous(expand = c(0,0), limits = c(0, NA))+
                    scale_y_continuous(expand = c(0,0), limits = c(0, NA))
lm_predictions
# This plots HDI ranks (y-axis) and average year of schooling (x-axis) to visualize the model prediction 
# as a straight line overlaid on the training dataset. It can be used in the further comparison with knn regression model. 

In [ ]:
# Extracts the RMPSE value from the table and returns a single numerical value.

lm_rmse <- lm_fit %>%
                predict(EYS_vs_HDI_Train) %>%
                bind_cols(EYS_vs_HDI_Train) %>%
                metrics(truth = HDI_Rank, estimate = .pred) %>%
                filter(.metric == 'rmse') %>%
                select(.estimate) %>%
                pull()
lm_rmse



In [ ]:
EYS_vs_HDI_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
                 set_engine("kknn") %>%
                 set_mode("regression")

# Sets the knn regression model specification.

EYS_vs_HDI_recipe <- recipe(HDI_Rank ~ Average_Years_of_Schooling, data = EYS_vs_HDI_Train) %>%
                   step_scale(all_predictors()) %>%
                   step_center(all_predictors())
EYS_vs_HDI_recipe

# Creates a recipe (HDI rank as the response variable and average year of schooling as the predictor) to preprocess our training data.

In [ ]:
EYS_vs_HDI_vfold <- vfold_cv(EYS_vs_HDI_Train, v = 5, strata = HDI_Rank)

EYS_vs_HDI_workflow <- workflow() %>%
                     add_recipe(EYS_vs_HDI_recipe) %>%
                     add_model(EYS_vs_HDI_spec)
EYS_vs_HDI_workflow

# Performs a cross-validation with five folds, establishes a workflow analysis and fits the knn regression model for our training dataset.

In [ ]:
gridvals <- tibble(neighbors = seq(1, 200))

# Creates a tibble with a sequence of values from 1 to 200 for running the cross validation.

EYS_vs_HDI_results <- EYS_vs_HDI_workflow %>%
                        tune_grid(resamples = EYS_vs_HDI_vfold, grid = gridvals) %>%
                        collect_metrics()
EYS_vs_HDI_results

# The resulting table contains important statistics such as k values ('neighbors' column), RMSPE values ('mean' column) 
# and the standard error which reflects how uncertain we are in the mean value.



In [ ]:
EYS_vs_HDI_min <- EYS_vs_HDI_results %>%
    filter(.metric == "rmse") %>%
    arrange(mean)  %>% 
    slice(1)
EYS_vs_HDI_min

# Finds the minimum RMSPE value to help us to find the best setting for the number of neighbors.

k_min <- EYS_vs_HDI_min %>%
            pull(neighbors)

# Extracts the best neighbors value.

In [ ]:
EYS_vs_HDI_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) %>%
                            set_engine("kknn") %>%
                            set_mode("regression")

# Sets the knn regression model specification with k_min.

EYS_vs_HDI_best_fit <- workflow() %>%
                        add_recipe(EYS_vs_HDI_recipe) %>%
                        add_model(EYS_vs_HDI_best_spec) %>%
                        fit(data = EYS_vs_HDI_Train)

# Establishes a workflow analysis with the new model specification and fits the knn regression model for our training dataset.

EYS_vs_HDI_summary <- EYS_vs_HDI_best_fit %>% 
                       predict(EYS_vs_HDI_Train) %>%
                       bind_cols(EYS_vs_HDI_Train) %>%
                       metrics(truth = HDI_Rank, estimate = .pred)
EYS_vs_HDI_summary

knn_rmse <- EYS_vs_HDI_summary %>%
                filter(.metric == 'rmse') %>%
                select(.estimate) %>%
                pull()

knn_rmse

In [ ]:
EYS_vs_HDI_summary_2 <- EYS_vs_HDI_best_fit %>% 
                       predict(EYS_vs_HDI_Test) %>%
                       bind_cols(EYS_vs_HDI_Test) %>%
                       metrics(truth = HDI_Rank, estimate = .pred)
EYS_vs_HDI_summary_2

knn_rmpse <- EYS_vs_HDI_summary_2 %>%
                filter(.metric == 'rmse') %>%
                select(.estimate) %>%
                pull()

knn_rmpse

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)


EYS_vs_HDI_plot <- ggplot(EYS_vs_HDI_Test, aes(x = Average_Years_of_Schooling, y = HDI_Rank)) +
            geom_point(alpha = 0.4) +
            labs(x= "Level of Education of an Individual (Average Years of Schooling)" , y = "HDI Rank of a Country (1 to 189)")+ 
            geom_line(data = EYS_vs_HDI_preds, aes(x = Average_Years_of_Schooling, y = .pred), color = "blue") +
            ggtitle(paste0("K = ", k_min)) +
            theme(text = element_text(size = 20))
EYS_vs_HDI_plot